# 기본 설정

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
import time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from xgboost import XGBClassifier
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
mrc_info = pd.read_csv("[Track1_데이터1] mrc_info.csv", encoding='euc-kr')
samp_train = pd.read_csv("[Track1_데이터2] samp_train.csv", encoding='euc-kr')
samp_cst_feat = pd.read_csv("[Track1_데이터3] samp_cst_feat.csv", encoding='euc-kr')
variable_dtype = pd.read_excel("[Track1_데이터4] variable_dtype.xlsx", encoding='euc-kr')

# 데이터 훑어보기

In [3]:
variable_dtype

,Variable_Name,dType
0,VAR002,numerical
1,VAR003,numerical
2,VAR004,numerical
3,VAR005,numerical
4,VAR006,numerical
5,VAR007,categorical
6,VAR008,numerical
7,VAR009,numerical
8,VAR010,numerical
9,VAR011,numerical


In [4]:
'''
MRC_ID_DI: 온라인 가맹점 번호
Category: 업종
'''
mrc_info

,MRC_ID_DI,Category
0,0,미이용
1,1,홈쇼핑
2,2,종합물
3,3,종합몰
4,4,전문몰
5,5,종합몰
6,6,홈쇼핑
7,7,오픈마켓_소셜
8,8,홈쇼핑
9,9,전문몰


In [5]:
'''
회원의 1월달 온라인 가맹점 이용여부 데이터
cst_id_di: 고객관리번호
MRC_ID_DI: 온라인 가맹점 번호 (0: 미이용)
'''
samp_train

,cst_id_di,MRC_ID_DI
0,90000000089,0
1,90000000176,8
2,90000000210,0
3,90000000212,5
4,90000000213,6
...,...,...
10119,90000460112,7
10120,90000460117,0
10121,90000460233,0
10122,90000460310,0


In [6]:
samp_cst_feat

,cst_id_di,VAR002,VAR003,VAR004,VAR005,VAR006,VAR007,VAR008,VAR009,VAR010,VAR011,VAR012,VAR013,VAR014,VAR015,VAR016,VAR017,VAR018,VAR019,VAR020,VAR021,VAR022,VAR023,VAR024,VAR025,VAR026,VAR027,VAR028,VAR029,VAR030,VAR031,VAR032,VAR033,VAR034,VAR035,VAR036,VAR037,VAR038,VAR039,VAR040,VAR041,VAR042,VAR043,VAR044,VAR045,VAR046,VAR047,VAR048,VAR049,VAR050,VAR051,VAR052,VAR053,VAR054,VAR055,VAR056,VAR057,VAR058,VAR059,VAR060,VAR061,VAR062,VAR063,VAR064,VAR065,VAR066,VAR067,VAR068,VAR069,VAR070,VAR071,VAR072,VAR073,VAR074,VAR075,VAR076,VAR077,VAR078,VAR079,VAR080,VAR081,VAR082,VAR083,VAR084,VAR085,VAR086,VAR087,VAR088,VAR089,VAR090,VAR091,VAR092,VAR093,VAR094,VAR095,VAR096,VAR097,VAR098,VAR099,VAR100,VAR101,VAR102,VAR103,VAR104,VAR105,VAR106,VAR107,VAR108,VAR109,VAR110,VAR111,VAR112,VAR113,VAR114,VAR115,VAR116,VAR117,VAR118,VAR119,VAR120,VAR121,VAR122,VAR123,VAR124,VAR125,VAR126,VAR127,VAR128,VAR129,VAR130,VAR131,VAR132,VAR133,VAR134,VAR135,VAR136,VAR137,VAR138,VAR139,VAR140,VAR141,VAR142,VAR143,VAR144,VAR145,VAR146,VAR147,VAR148,VAR149,VAR150,VAR151,VAR152,VAR153,VAR154,VAR155,VAR156,VAR157,VAR158,VAR159,VAR160,VAR161,VAR162,VAR163,VAR164,VAR165,VAR166,VAR167,VAR168,VAR169,VAR170,VAR171,VAR172,VAR173,VAR174,VAR175,VAR176,VAR177,var178,VAR179,VAR180,VAR181,VAR182,VAR183,VAR184,VAR185,VAR186,VAR187,VAR188,VAR189,VAR190,VAR191,VAR192,VAR193,VAR194,VAR195,VAR196,VAR197,VAR198,VAR199,VAR200,VAR201,VAR202,VAR203,VAR204,VAR205,VAR206,VAR207,VAR208,VAR209,VAR210,VAR211,VAR212,VAR213,VAR214,VAR215,VAR216,VAR217,VAR218,VAR219,VAR220,VAR221,VAR222,VAR223,VAR224,VAR225,VAR226,VAR227
0,90000000089,-0.06610,0.5280,-0.13607,0.10945,0.06557,0,0.7702,-0.18965,0.1981,0.24149,0.32429,-0.18611,0.06960,1,0.0049,0.8085,0,-0.11899,0.03969,-0.16651,0.12037,-0.01715,-0.09941,-0.28878,0,-0.07481,-0.12742,0.14367,0.06725,0.7702,0.04138,-0.01619,-0.32429,0.11580,0.7179,0.16920,-0.08048,0.28126,-0.11289,-0.02092,0.5381,0.22047,0.17756,0.03794,-0.04432,-0.28743,-0.01164,0.26300,-0.15466,-0.08374,-0.12935,-0.24726,-0.32134,0.39311,-0.32813,0.35632,0.12739,1,-0.18338,-0.07717,0.4688,0.18707,-0.03818,-0.04869,1,1,-0.14361,0.20281,0,0,0.02536,-0.20705,0.03747,0.28517,-0.18100,0,0,-0.20069,0.23156,-0.18709,0.20059,0.04329,0.0025,0.13105,-0.00891,0.23236,0.10216,0.28202,0.0077,-0.09886,0.29902,-0.22713,0,-0.23064,0,0,0,0.06628,0.11613,0.0093,0.03583,0.3578,0.20585,1.0000,-0.29631,0,0.0008,0.26873,0.05674,0,0.02082,0.26616,-0.25601,0.16075,0.15089,0.7286,0.07524,-0.11761,-0.13649,0.16103,0.38476,0.18881,0,0.00524,0.29896,1,0.36885,0.05585,0.06960,0.0068,0.05439,-0.04895,0.22997,0.06607,-0.02500,-0.05500,0.0020,0.07158,-0.11915,-0.09522,0.11580,1,0,0,0.35606,-0.32429,0,0.19451,-0.22885,0.01926,-0.15068,0.13834,-0.16525,0.09025,0.01253,0.20219,0.31263,-0.17855,-0.20515,-0.18453,-0.21851,0.37549,-0.10744,0,0.13043,0.0013,-0.17485,0.18986,-0.07305,0.0069,-0.30157,-0.16382,0.28385,-0.14273,-0.27069,0,0.3766,0,0.06165,0.0021,-0.03891,0.04840,-0.14790,0.26533,-0.00426,0.14765,-0.13332,0.03077,0.23017,0.11764,0.5740,0.06403,0.06184,0.04242,-0.04322,0.0051,0.04523,0,0.07247,0.09631,-0.22864,0.07262,-0.08837,0.01070,0.06340,0.12424,0,0.20392,0.24685,0.01432,-0.00966,0.09746,-0.17616,-0.18956,-0.13676,-0.13279,-0.32829,0.19113,0.05449,0.09471,0.27091,0.01931,0.02938,0.17105,0.12537,0.22197
1,90000000176,-0.09537,0.1347,-0.13541,0.17331,-0.19657,0,0.0616,-0.23104,0.4940,-0.39476,-0.38366,0.39329,0.17549,0,0.3130,0.6564,0,0.31544,0.03794,-0.30237,-0.34538,0.04368,0.04381,-0.19160,0,-0.09801,-0.04593,0.11228,0.11692,0.5661,0.19938,0.01259,-0.17976,0.20081,0.7454,0.24256,0.01363,-0.22807,0.06728,0.17312,0.5369,0.23387,0.04572,0.05406,-0.18782,-0.23752,-0.10148,-0.16942,0.02893,0.03702,0.04855,0.36531,0.24302,-0.30677,-0.20810,-0.12376,0.15054,0,0.22430,0.10513,0.3797,-0.16796,0.01456,-0.10743,0,1,-0.08246,-0.38202,0,0,0.02066,0.28113,0.05687,-0.11689,0.16837,1,1,-0.08043,0.21618,-0.20256,0.23597,0.06233,0.3285,0.05898,0.00544,-0.29231,0.02483,-0.38122,0.0065,0.39975,0.09775,-0.34015

# 방법 구상

> 1st
- samp_cst_feat t-sne로 압축해서 시각화해본 다음에 samp_train class랑 비교해보기
- class 중에서 미이용, 종합몰, 홈쇼핑, 전문몰, 오픈마켓 5개로도 비교해보기

> 2nd
- samp_train을 8:2로 나눠서 실제 모델링을 해보고 그것을 바탕으로 score 내보기

# <font color="red">2. 샘플 데이터로 모델 돌려보기</font>

### 데이터 다듬기

In [7]:
df = samp_cst_feat.merge(samp_train, how="left", on="cst_id_di")
df = df.merge(mrc_info, how='left', on="MRC_ID_DI")
df = df[['cst_id_di','MRC_ID_DI', 'Category'] + list(df.columns)[1:-2]]
df

,cst_id_di,MRC_ID_DI,Category,VAR002,VAR003,VAR004,VAR005,VAR006,VAR007,VAR008,VAR009,VAR010,VAR011,VAR012,VAR013,VAR014,VAR015,VAR016,VAR017,VAR018,VAR019,VAR020,VAR021,VAR022,VAR023,VAR024,VAR025,VAR026,VAR027,VAR028,VAR029,VAR030,VAR031,VAR032,VAR033,VAR034,VAR035,VAR036,VAR037,VAR038,VAR039,VAR040,VAR041,VAR042,VAR043,VAR044,VAR045,VAR046,VAR047,VAR048,VAR049,VAR050,VAR051,VAR052,VAR053,VAR054,VAR055,VAR056,VAR057,VAR058,VAR059,VAR060,VAR061,VAR062,VAR063,VAR064,VAR065,VAR066,VAR067,VAR068,VAR069,VAR070,VAR071,VAR072,VAR073,VAR074,VAR075,VAR076,VAR077,VAR078,VAR079,VAR080,VAR081,VAR082,VAR083,VAR084,VAR085,VAR086,VAR087,VAR088,VAR089,VAR090,VAR091,VAR092,VAR093,VAR094,VAR095,VAR096,VAR097,VAR098,VAR099,VAR100,VAR101,VAR102,VAR103,VAR104,VAR105,VAR106,VAR107,VAR108,VAR109,VAR110,VAR111,VAR112,VAR113,VAR114,VAR115,VAR116,VAR117,VAR118,VAR119,VAR120,VAR121,VAR122,VAR123,VAR124,VAR125,VAR126,VAR127,VAR128,VAR129,VAR130,VAR131,VAR132,VAR133,VAR134,VAR135,VAR136,VAR137,VAR138,VAR139,VAR140,VAR141,VAR142,VAR143,VAR144,VAR145,VAR146,VAR147,VAR148,VAR149,VAR150,VAR151,VAR152,VAR153,VAR154,VAR155,VAR156,VAR157,VAR158,VAR159,VAR160,VAR161,VAR162,VAR163,VAR164,VAR165,VAR166,VAR167,VAR168,VAR169,VAR170,VAR171,VAR172,VAR173,VAR174,VAR175,VAR176,VAR177,var178,VAR179,VAR180,VAR181,VAR182,VAR183,VAR184,VAR185,VAR186,VAR187,VAR188,VAR189,VAR190,VAR191,VAR192,VAR193,VAR194,VAR195,VAR196,VAR197,VAR198,VAR199,VAR200,VAR201,VAR202,VAR203,VAR204,VAR205,VAR206,VAR207,VAR208,VAR209,VAR210,VAR211,VAR212,VAR213,VAR214,VAR215,VAR216,VAR217,VAR218,VAR219,VAR220,VAR221,VAR222,VAR223,VAR224,VAR225,VAR226,VAR227
0,90000000089,0,미이용,-0.06610,0.5280,-0.13607,0.10945,0.06557,0,0.7702,-0.18965,0.1981,0.24149,0.32429,-0.18611,0.06960,1,0.0049,0.8085,0,-0.11899,0.03969,-0.16651,0.12037,-0.01715,-0.09941,-0.28878,0,-0.07481,-0.12742,0.14367,0.06725,0.7702,0.04138,-0.01619,-0.32429,0.11580,0.7179,0.16920,-0.08048,0.28126,-0.11289,-0.02092,0.5381,0.22047,0.17756,0.03794,-0.04432,-0.28743,-0.01164,0.26300,-0.15466,-0.08374,-0.12935,-0.24726,-0.32134,0.39311,-0.32813,0.35632,0.12739,1,-0.18338,-0.07717,0.4688,0.18707,-0.03818,-0.04869,1,1,-0.14361,0.20281,0,0,0.02536,-0.20705,0.03747,0.28517,-0.18100,0,0,-0.20069,0.23156,-0.18709,0.20059,0.04329,0.0025,0.13105,-0.00891,0.23236,0.10216,0.28202,0.0077,-0.09886,0.29902,-0.22713,0,-0.23064,0,0,0,0.06628,0.11613,0.0093,0.03583,0.3578,0.20585,1.0000,-0.29631,0,0.0008,0.26873,0.05674,0,0.02082,0.26616,-0.25601,0.16075,0.15089,0.7286,0.07524,-0.11761,-0.13649,0.16103,0.38476,0.18881,0,0.00524,0.29896,1,0.36885,0.05585,0.06960,0.0068,0.05439,-0.04895,0.22997,0.06607,-0.02500,-0.05500,0.0020,0.07158,-0.11915,-0.09522,0.11580,1,0,0,0.35606,-0.32429,0,0.19451,-0.22885,0.01926,-0.15068,0.13834,-0.16525,0.09025,0.01253,0.20219,0.31263,-0.17855,-0.20515,-0.18453,-0.21851,0.37549,-0.10744,0,0.13043,0.0013,-0.17485,0.18986,-0.07305,0.0069,-0.30157,-0.16382,0.28385,-0.14273,-0.27069,0,0.3766,0,0.06165,0.0021,-0.03891,0.04840,-0.14790,0.26533,-0.00426,0.14765,-0.13332,0.03077,0.23017,0.11764,0.5740,0.06403,0.06184,0.04242,-0.04322,0.0051,0.04523,0,0.07247,0.09631,-0.22864,0.07262,-0.08837,0.01070,0.06340,0.12424,0,0.20392,0.24685,0.01432,-0.00966,0.09746,-0.17616,-0.18956,-0.13676,-0.13279,-0.32829,0.19113,0.05449,0.09471,0.27091,0.01931,0.02938,0.17105,0.12537,0.22197
1,90000000176,8,홈쇼핑,-0.09537,0.1347,-0.13541,0.17331,-0.19657,0,0.0616,-0.23104,0.4940,-0.39476,-0.38366,0.39329,0.17549,0,0.3130,0.6564,0,0.31544,0.03794,-0.30237,-0.34538,0.04368,0.04381,-0.19160,0,-0.09801,-0.04593,0.11228,0.11692,0.5661,0.19938,0.01259,-0.17976,0.20081,0.7454,0.24256,0.01363,-0.22807,0.06728,0.17312,0.5369,0.23387,0.04572,0.05406,-0.18782,-0.23752,-0.10148,-0.16942,0.02893,0.03702,0.04855,0.36531,0.24302,-0.30677,-0.20810,-0.12376,0.15054,0,0.22430,0.10513,0.3797,-0.16796,0.01456,-0.10743,0,1,-0.08246,-0.38202,0,0,0.02066,0.28113,0.05687,-0.11689,0.16837,1,1,-0.08043,0.21618,-0.20256,0.23597,0.06233,0.3285,0.05898,0.00544,-0.29231,0.02483,-0.38122,

### Voting

In [8]:
X = df.iloc[:, 3:].values
y = df.iloc[:, 1].values

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [10]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8099, 226), (2025, 226), (8099,), (2025,))

xgboost

In [11]:
## Xgboost Classifier
xgb1 = XGBClassifier(learning_rate=0.01, max_depth=30,random_state=42,
                     min_child_weight=1, missing=None, n_estimators=200, n_jobs=-1,
                     silent=None, subsample=0.9, verbosity=1)

In [12]:
## Xgboost Classifier
xgb2 = XGBClassifier(learning_rate=0.02, max_depth=20, random_state=42,
                     min_child_weight=1, missing=None, n_estimators=150, n_jobs=-1,
                     silent=None, subsample=0.9, verbosity=1)

lgbm

In [13]:
lg = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None,
                        colsample_bytree=0.9234, learning_rate=0.01, max_depth=20, 
                        min_child_weight=0.1, min_split_gain=0.0, n_estimators=150, 
                        n_jobs=-1, num_leaves=13, random_state=42, silent=True)

catboost

In [14]:
catbo = CatBoostClassifier(depth=15, iterations= 170, l2_leaf_reg= 9, learning_rate= 0.05)

voting

In [15]:
voting_clf = VotingClassifier(
    estimators=[('xgb1', xgb1), ('xgb2', xgb2), ('lgbm', lg), ('cat', catbo)],
    voting='soft')

catboost 전까지는 학습에 대략 20분 걸림

In [17]:
start = time.time()

voting_clf.fit(X_train, y_train)

finished_time_fit = time.time() - start

0:	learn: 2.2967778	total: 3m 9s	remaining: 8h 52m 49s
1:	learn: 2.1780060	total: 6m 9s	remaining: 8h 37m 1s
2:	learn: 2.0665631	total: 9m 10s	remaining: 8h 30m 27s
3:	learn: 1.9706788	total: 11m 58s	remaining: 8h 16m 53s
4:	learn: 1.8794445	total: 14m 59s	remaining: 8h 14m 52s
5:	learn: 1.7964865	total: 18m 2s	remaining: 8h 12m 57s
6:	learn: 1.7342841	total: 21m 1s	remaining: 8h 9m 33s
7:	learn: 1.6717694	total: 23m 47s	remaining: 8h 1m 53s
8:	learn: 1.6215691	total: 26m 49s	remaining: 7h 59m 55s
9:	learn: 1.5857569	total: 29m 47s	remaining: 7h 56m 37s
10:	learn: 1.5353979	total: 32m 41s	remaining: 7h 52m 27s
11:	learn: 1.4889937	total: 35m 37s	remaining: 7h 48m 59s
12:	learn: 1.4536953	total: 38m 35s	remaining: 7h 46m 5s
13:	learn: 1.4299931	total: 41m 38s	remaining: 7h 43m 57s
14:	learn: 1.3827366	total: 44m 40s	remaining: 7h 41m 43s
15:	learn: 1.3621830	total: 47m 42s	remaining: 7h 39m 9s
16:	learn: 1.3384259	total: 50m 48s	remaining: 7h 37m 14s
17:	learn: 1.3272965	total: 53m 49s	

137:	learn: 0.6178874	total: 6h 18m 19s	remaining: 1h 27m 43s
138:	learn: 0.6159522	total: 6h 20m 51s	remaining: 1h 24m 56s
139:	learn: 0.6137162	total: 6h 23m 33s	remaining: 1h 22m 11s
140:	learn: 0.6115797	total: 6h 26m 14s	remaining: 1h 19m 26s
141:	learn: 0.6096135	total: 6h 28m 58s	remaining: 1h 16m 41s
142:	learn: 0.6076239	total: 6h 31m 44s	remaining: 1h 13m 57s
143:	learn: 0.6057338	total: 6h 34m 16s	remaining: 1h 11m 11s
144:	learn: 0.6040170	total: 6h 36m 56s	remaining: 1h 8m 26s
145:	learn: 0.6023054	total: 6h 39m 37s	remaining: 1h 5m 41s
146:	learn: 0.6003574	total: 6h 42m 16s	remaining: 1h 2m 56s
147:	learn: 0.5982244	total: 6h 44m 58s	remaining: 1h 11s
148:	learn: 0.5964265	total: 6h 47m 43s	remaining: 57m 27s
149:	learn: 0.5946395	total: 6h 50m 29s	remaining: 54m 43s
150:	learn: 0.5929126	total: 6h 53m 12s	remaining: 51m 59s
151:	learn: 0.5913209	total: 6h 54m 53s	remaining: 49m 7s
152:	learn: 0.5895960	total: 6h 57m 40s	remaining: 46m 24s
153:	learn: 0.5880302	total: 7h

In [18]:
finished_time_fit

28896.163578033447

In [19]:
start = time.time()

y_pred = voting_clf.predict(X_test)

finished_time_prediction = time.time() - start
finished_time_prediction

1.6907298564910889

In [20]:
f1_score(y_test, y_pred, average='weighted')

0.7213305826483624

모델 저장하기

In [22]:
import _pickle as cPickle


In [23]:
with open('voting_classifier ver1.0.pkl', 'wb') as fid:
    cPickle.dump(voting_clf, fid)    

In [24]:
# load it again
with open('voting_classifier ver1.0.pkl', 'rb') as fid:
    loaded_voting_model = cPickle.load(fid)

In [25]:
y_pred = loaded_voting_model.predict(X_test)
f1_score(y_test, y_pred, average='weighted')

0.7213305826483624

### 11 class classification (XGboost)

In [7]:
df = samp_cst_feat.merge(samp_train, how="left", on="cst_id_di")
df = df.merge(mrc_info, how='left', on="MRC_ID_DI")
df = df[['cst_id_di','MRC_ID_DI', 'Category'] + list(df.columns)[1:-2]]
df

,cst_id_di,MRC_ID_DI,Category,VAR002,VAR003,VAR004,VAR005,VAR006,VAR007,VAR008,VAR009,VAR010,VAR011,VAR012,VAR013,VAR014,VAR015,VAR016,VAR017,VAR018,VAR019,VAR020,VAR021,VAR022,VAR023,VAR024,VAR025,VAR026,VAR027,VAR028,VAR029,VAR030,VAR031,VAR032,VAR033,VAR034,VAR035,VAR036,VAR037,VAR038,VAR039,VAR040,VAR041,VAR042,VAR043,VAR044,VAR045,VAR046,VAR047,VAR048,VAR049,VAR050,VAR051,VAR052,VAR053,VAR054,VAR055,VAR056,VAR057,VAR058,VAR059,VAR060,VAR061,VAR062,VAR063,VAR064,VAR065,VAR066,VAR067,VAR068,VAR069,VAR070,VAR071,VAR072,VAR073,VAR074,VAR075,VAR076,VAR077,VAR078,VAR079,VAR080,VAR081,VAR082,VAR083,VAR084,VAR085,VAR086,VAR087,VAR088,VAR089,VAR090,VAR091,VAR092,VAR093,VAR094,VAR095,VAR096,VAR097,VAR098,VAR099,VAR100,VAR101,VAR102,VAR103,VAR104,VAR105,VAR106,VAR107,VAR108,VAR109,VAR110,VAR111,VAR112,VAR113,VAR114,VAR115,VAR116,VAR117,VAR118,VAR119,VAR120,VAR121,VAR122,VAR123,VAR124,VAR125,VAR126,VAR127,VAR128,VAR129,VAR130,VAR131,VAR132,VAR133,VAR134,VAR135,VAR136,VAR137,VAR138,VAR139,VAR140,VAR141,VAR142,VAR143,VAR144,VAR145,VAR146,VAR147,VAR148,VAR149,VAR150,VAR151,VAR152,VAR153,VAR154,VAR155,VAR156,VAR157,VAR158,VAR159,VAR160,VAR161,VAR162,VAR163,VAR164,VAR165,VAR166,VAR167,VAR168,VAR169,VAR170,VAR171,VAR172,VAR173,VAR174,VAR175,VAR176,VAR177,var178,VAR179,VAR180,VAR181,VAR182,VAR183,VAR184,VAR185,VAR186,VAR187,VAR188,VAR189,VAR190,VAR191,VAR192,VAR193,VAR194,VAR195,VAR196,VAR197,VAR198,VAR199,VAR200,VAR201,VAR202,VAR203,VAR204,VAR205,VAR206,VAR207,VAR208,VAR209,VAR210,VAR211,VAR212,VAR213,VAR214,VAR215,VAR216,VAR217,VAR218,VAR219,VAR220,VAR221,VAR222,VAR223,VAR224,VAR225,VAR226,VAR227
0,90000000089,0,미이용,-0.06610,0.5280,-0.13607,0.10945,0.06557,0,0.7702,-0.18965,0.1981,0.24149,0.32429,-0.18611,0.06960,1,0.0049,0.8085,0,-0.11899,0.03969,-0.16651,0.12037,-0.01715,-0.09941,-0.28878,0,-0.07481,-0.12742,0.14367,0.06725,0.7702,0.04138,-0.01619,-0.32429,0.11580,0.7179,0.16920,-0.08048,0.28126,-0.11289,-0.02092,0.5381,0.22047,0.17756,0.03794,-0.04432,-0.28743,-0.01164,0.26300,-0.15466,-0.08374,-0.12935,-0.24726,-0.32134,0.39311,-0.32813,0.35632,0.12739,1,-0.18338,-0.07717,0.4688,0.18707,-0.03818,-0.04869,1,1,-0.14361,0.20281,0,0,0.02536,-0.20705,0.03747,0.28517,-0.18100,0,0,-0.20069,0.23156,-0.18709,0.20059,0.04329,0.0025,0.13105,-0.00891,0.23236,0.10216,0.28202,0.0077,-0.09886,0.29902,-0.22713,0,-0.23064,0,0,0,0.06628,0.11613,0.0093,0.03583,0.3578,0.20585,1.0000,-0.29631,0,0.0008,0.26873,0.05674,0,0.02082,0.26616,-0.25601,0.16075,0.15089,0.7286,0.07524,-0.11761,-0.13649,0.16103,0.38476,0.18881,0,0.00524,0.29896,1,0.36885,0.05585,0.06960,0.0068,0.05439,-0.04895,0.22997,0.06607,-0.02500,-0.05500,0.0020,0.07158,-0.11915,-0.09522,0.11580,1,0,0,0.35606,-0.32429,0,0.19451,-0.22885,0.01926,-0.15068,0.13834,-0.16525,0.09025,0.01253,0.20219,0.31263,-0.17855,-0.20515,-0.18453,-0.21851,0.37549,-0.10744,0,0.13043,0.0013,-0.17485,0.18986,-0.07305,0.0069,-0.30157,-0.16382,0.28385,-0.14273,-0.27069,0,0.3766,0,0.06165,0.0021,-0.03891,0.04840,-0.14790,0.26533,-0.00426,0.14765,-0.13332,0.03077,0.23017,0.11764,0.5740,0.06403,0.06184,0.04242,-0.04322,0.0051,0.04523,0,0.07247,0.09631,-0.22864,0.07262,-0.08837,0.01070,0.06340,0.12424,0,0.20392,0.24685,0.01432,-0.00966,0.09746,-0.17616,-0.18956,-0.13676,-0.13279,-0.32829,0.19113,0.05449,0.09471,0.27091,0.01931,0.02938,0.17105,0.12537,0.22197
1,90000000176,8,홈쇼핑,-0.09537,0.1347,-0.13541,0.17331,-0.19657,0,0.0616,-0.23104,0.4940,-0.39476,-0.38366,0.39329,0.17549,0,0.3130,0.6564,0,0.31544,0.03794,-0.30237,-0.34538,0.04368,0.04381,-0.19160,0,-0.09801,-0.04593,0.11228,0.11692,0.5661,0.19938,0.01259,-0.17976,0.20081,0.7454,0.24256,0.01363,-0.22807,0.06728,0.17312,0.5369,0.23387,0.04572,0.05406,-0.18782,-0.23752,-0.10148,-0.16942,0.02893,0.03702,0.04855,0.36531,0.24302,-0.30677,-0.20810,-0.12376,0.15054,0,0.22430,0.10513,0.3797,-0.16796,0.01456,-0.10743,0,1,-0.08246,-0.38202,0,0,0.02066,0.28113,0.05687,-0.11689,0.16837,1,1,-0.08043,0.21618,-0.20256,0.23597,0.06233,0.3285,0.05898,0.00544,-0.29231,0.02483,-0.38122,

In [8]:
X = df.iloc[:, 3:].values
y = df.iloc[:, 1].values

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [10]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8099, 226), (2025, 226), (8099,), (2025,))

In [11]:
## Xgboost Classifier
model = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
      colsample_bynode=1, colsample_bytree=0.9, gamma=0,
      learning_rate=0.02, max_delta_step=0, max_depth=20,
      min_child_weight=1, missing=None, n_estimators=150, n_jobs=1,
      nthread=None, objective='binary:logistic', random_state=0,
      reg_alpha=0, reg_lambda=1, seed=None,
      silent=None, subsample=0.9, verbosity=1)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.9, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.02, max_delta_step=0, max_depth=20,
              min_child_weight=1, missing=None, monotone_constraints='()',
              n_estimators=150, n_jobs=1, nthread=1, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, seed=0, silent=None,
              subsample=0.9, tree_method='exact', validate_parameters=1,
              verbosity=1)

In [12]:
model.predict(X_test)
y_pred = model.predict(X_test)
f1_score(y_test,y_pred, average='weighted')

0.7391742741228138

## 이후에 할 것들

> 모델링
- Ensemble로 ensemble하기 (현우)
- SMOTE해서 해야할 것들 (현우)
- (완료) 0번들 drop하고 t-sne
- 모델을 2단계로 나눠서 해볼까? (형준)
- Clustering Silhoutte score 구해보기 (유진)
- PCA하고서 모델링 / 다중공선성 / Outlier detection 제거하고 PCA하고 모델링 (option)

> 마케팅
- 타겟팅 쿠폰
- 교수님 SOS (현우)
- 회사 정보 뒤지기 (공동)
- 논문 뒤지기 (공동)
- 판매자 5개 카테고리별로 도메인 knowledge 찾아보기 (공동)
- 마케팅에 추가 데이터 쓰기